In [1]:
from google.cloud import bigquery

In [2]:
import pandas

In [16]:
GCP_CREDENTIALS = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
GCP_CREDENTIALS

In [17]:
# API BigQuery
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=GCP_CREDENTIALS)

In [44]:
QUERY = """

INSERT INTO `peya-data-qlty-pro.gcp_cost_control.usage_metrics_per_project_daily_agg` 
                (execution_date, 
                project_id, 
                account_type,
                total_queries,
                total_processed_tb, 
                total_costs_usd, 
                total_duration_in_seconds, 
                avg_duration_in_seconds,
                total_slot_usage,
                avg_slot_usage,
                total_users,
                _audi_load_date)
            SELECT  
                execution_date ,
                project_id, 
                account_type,
                sum( total_queries ) as total_queries,
                sum(total_processed_tb) as total_processed_tb, 
                sum(total_costs_usd) as total_costs_usd,
                sum( total_duration_in_seconds) as total_duration_in_seconds,
                avg(total_duration_in_seconds) as avg_duration_in_seconds,
                sum( slot_usage ) as total_slot_usage,
                avg( slot_usage ) as avg_slot_usage,
                count(distinct user_email) as total_users,
                CURRENT_TIMESTAMP() as _audi_load_date
            FROM 
                `peya-data-qlty-pro.gcp_cost_control.usage_metrics_per_user_daily_agg` 
            WHERE 
                execution_date >= "2020-12-01"
                AND project_id = "peya-data-qlty-pro"
            GROUP BY 
                execution_date, 
                project_id,
                account_type
            ORDER BY 
                execution_date, 
                Project_id, 
                account_type


"""

In [45]:
query_job = bq_client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [ ]:
rows.to_dataframe()